# ProfileScope: Social Media Profile Analysis Demo

This notebook demonstrates how to use ProfileScope to analyze social media profiles and interpret the results.

## Setup and Installation

First, let's import the required modules and set up our environment.

In [ ]:
import sys
import os
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the project root to the path so we can import our modules
sys.path.append(os.path.abspath('..'))

# Import ProfileScope modules
from app.core.analyzer import SocialMediaAnalyzer

# Set plot style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## Basic Usage: Simple Test

Let's create a simple example to verify that our analyzer works properly:

In [ ]:
# Initialize the analyzer
analyzer = SocialMediaAnalyzer()

# Print config to verify
print("Analyzer configuration:")
print(json.dumps(analyzer.config, indent=2))

In [ ]:
# Define a test profile
platform = "twitter"
profile_id = "test_user"

# Perform a test analysis
results = analyzer.analyze_profile(platform, profile_id)

# Print results summary
print(f"Analysis completed for {profile_id} on {platform}")
print(f"Analysis date: {results['metadata']['analysis_date']}")
print(f"Analyzer version: {results['metadata']['analyzer_version']}")

## Exploring the Results

Now let's explore what's in the analysis results:

In [ ]:
# Let's see what top-level keys are in the results
print("Results structure:")
for key in results.keys():
    print(f"- {key}")

## Examining UI Tab Components

In the desktop application, we have three main tabs to present the analysis results. Let's look at how each tab is structured.

### Summary Tab

The Summary tab gives an overview of the profile analysis with key metrics.

In [ ]:
# Let's look at what's in the content_analysis section
if 'content_analysis' in results:
    print("Content Analysis Keys:")
    for key in results['content_analysis'].keys():
        print(f"- {key}")

    # Print a sample of profile summary
    if 'profile_summary' in results['content_analysis']:
        print("\nProfile Summary:")
        print(results['content_analysis']['profile_summary'])

    # Activity metrics - important for the Summary tab
    if 'activity_metrics' in results['content_analysis']:
        print("\nActivity Metrics:")
        for metric, value in results['content_analysis']['activity_metrics'].items():
            print(f"{metric}: {value}")

### Authenticity Tab

The Authenticity tab shows metrics related to the authenticity analysis of the profile. It includes an authenticity score, consistency metrics, and potential risk factors.

In [ ]:
# Explore the authenticity analysis data
if 'authenticity_analysis' in results:
    auth = results['authenticity_analysis']
    print("Authenticity Analysis Keys:")
    for key in auth.keys():
        print(f"- {key}")

    # Overall authenticity
    if 'overall_authenticity' in auth:
        score = auth['overall_authenticity']['score']
        confidence = auth['overall_authenticity'].get('confidence', 0)
        print(f"\nAuthenticity Score: {score:.2f} (Confidence: {confidence:.2f})")
        
        # Create a simple authenticity gauge visualization
        plt.figure(figsize=(10, 2))
        plt.barh(y=['Authenticity'], width=[score], color='green', height=0.4)
        plt.barh(y=['Authenticity'], width=[1], color='lightgray', height=0.4, alpha=0.3)
        plt.xlim(0, 1)
        plt.title('Authenticity Score')
        plt.show()

    # Risk factors
    if 'risk_factors' in auth:
        print("\nRisk Factors:")
        for factor in auth['risk_factors']:
            print(f"- {factor['factor']}: {factor['severity']} (Confidence: {factor['confidence']})")

### Predictions Tab

The Predictions tab displays forecasts and predictions about the profile based on the analysis. This includes future interests, potential behaviors, and demographic predictions.

In [ ]:
# Explore the predictions data
if 'predictions' in results:
    pred = results['predictions']
    print("Predictions Keys:")
    for key in pred.keys():
        print(f"- {key}")
    
    # Future interests
    if 'future_interests' in pred and pred['future_interests']:
        print("\nPredicted Future Interests:")
        for interest in pred['future_interests']:
            interest_name = interest['interest']
            confidence = interest['confidence']
            reasoning = interest.get('reasoning', 'No reasoning provided')
            print(f"- {interest_name} (Confidence: {confidence:.2f})")
            print(f"  Reasoning: {reasoning}")
    
    # Demographics
    if 'demographic_predictions' in pred:
        print("\nDemographic Predictions:")
        demog = pred['demographic_predictions']
        for category, details in demog.items():
            if isinstance(details, dict) and 'prediction' in details:
                print(f"- {category.replace('_', ' ').title()}: {details['prediction']} (Confidence: {details.get('confidence', 0):.2f})")

    # Alternative naming patterns: career paths
    if 'career_paths' in pred and pred['career_paths']:
        print("\nPredicted Career Paths:")
        for career in pred['career_paths']:
            path = career['path']
            likelihood = career.get('likelihood', 0)
            print(f"- {path} (Likelihood: {likelihood:.2f})")
            if 'justification' in career:
                print(f"  Justification: {career['justification']}")

    # Brand affinities (alternative naming)
    if 'brand_affinities' in pred:
        print("\nBrand Affinities:")
        for category, brands in pred['brand_affinities'].items():
            if isinstance(brands, list) and brands:
                print(f"- {category.replace('_', ' ').title()}: {', '.join(brands)}")

## Desktop Application Visualization

The ProfileScope desktop application presents all this information in a user-friendly interface with three main tabs:

In [ ]:
# Mock visualization of how our data is presented in the desktop app
from IPython.display import HTML, display

def show_desktop_app_preview():
    html = """
    <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px;">
        <h3 style="text-align: center;">ProfileScope Desktop Application</h3>
        <div style="display: flex; justify-content: space-around; margin-bottom: 10px;">
            <div style="background-color: #007bff; color: white; padding: 5px 20px; border-radius: 5px;">Summary Tab</div>
            <div style="background-color: #6c757d; color: white; padding: 5px 20px; border-radius: 5px;">Authenticity Tab</div>
            <div style="background-color: #6c757d; color: white; padding: 5px 20px; border-radius: 5px;">Predictions Tab</div>
        </div>
        <div style="background-color: white; padding: 15px; border-radius: 5px;">
            <h4>Profile Summary</h4>
            <p>User analysis of test_user on twitter platform. The profile shows regular activity with mixed sentiment.</p>
            <div style="display: flex; justify-content: space-between;">
                <div style="flex: 1; margin-right: 10px;">
                    <h5>Key Metrics</h5>
                    <ul>
                        <li><b>Posts:</b> 342</li>
                        <li><b>Activity Level:</b> Medium</li>
                        <li><b>Post Frequency:</b> 3.2 per week</li>
                    </ul>
                </div>
                <div style="flex: 1;">
                    <h5>Sentiment Analysis</h5>
                    <div style="background-color: #e9ecef; height: 20px; width: 100%; border-radius: 10px;">
                        <div style="background-color: #28a745; height: 100%; width: 65%; border-radius: 10px;"></div>
                    </div>
                    <p><small>65% Positive</small></p>
                </div>
            </div>
        </div>
    </div>
    """
    display(HTML(html))

show_desktop_app_preview()

In [ ]:
# Show Authenticity Tab preview
def show_authenticity_preview():
    html = """
    <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px;">
        <h3 style="text-align: center;">ProfileScope Desktop Application</h3>
        <div style="display: flex; justify-content: space-around; margin-bottom: 10px;">
            <div style="background-color: #6c757d; color: white; padding: 5px 20px; border-radius: 5px;">Summary Tab</div>
            <div style="background-color: #007bff; color: white; padding: 5px 20px; border-radius: 5px;">Authenticity Tab</div>
            <div style="background-color: #6c757d; color: white; padding: 5px 20px; border-radius: 5px;">Predictions Tab</div>
        </div>
        <div style="background-color: white; padding: 15px; border-radius: 5px;">
            <div style="background-color: #fff3cd; color: #856404; padding: 10px; border-radius: 5px; margin-bottom: 15px;">
                ⚠️ This is mock data for demonstration purposes only.
            </div>
            
            <h4>Authenticity Assessment</h4>
            
            <!-- Authenticity Gauge -->
            <div style="text-align: center; margin-bottom: 20px;">
                <div style="position: relative; width: 200px; height: 100px; margin: auto; overflow: hidden;">
                    <div style="position: absolute; width: 200px; height: 200px; border-radius: 50%; background: conic-gradient(green 0% 85%, red 85% 100%); top: 0;"></div>
                    <div style="position: absolute; width: 160px; height: 160px; border-radius: 50%; background: white; top: 20px; left: 20px;"></div>
                    <div style="position: absolute; width: 100%; text-align: center; top: 70px;">85%</div>
                </div>
                <p><b>Authenticity Score: 85%</b> (High)</p>
            </div>
            
            <div style="display: flex; justify-content: space-between;">
                <div style="flex: 1; margin-right: 10px;">
                    <h5>Assessment Summary</h5>
                    <p>This profile demonstrates consistent behavior patterns and writing style across its history. Few irregularities detected.</p>
                </div>
                <div style="flex: 1;">
                    <h5>Risk Factors</h5>
                    <ul>
                        <li>Unusual activity spike (Low severity)</li>
                        <li>Minor writing style inconsistencies (Very low severity)</li>
                    </ul>
                </div>
            </div>
        </div>
    </div>
    """
    display(HTML(html))

show_authenticity_preview()

In [ ]:
# Show Predictions Tab preview
def show_predictions_preview():
    html = """
    <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px;">
        <h3 style="text-align: center;">ProfileScope Desktop Application</h3>
        <div style="display: flex; justify-content: space-around; margin-bottom: 10px;">
            <div style="background-color: #6c757d; color: white; padding: 5px 20px; border-radius: 5px;">Summary Tab</div>
            <div style="background-color: #6c757d; color: white; padding: 5px 20px; border-radius: 5px;">Authenticity Tab</div>
            <div style="background-color: #007bff; color: white; padding: 5px 20px; border-radius: 5px;">Predictions Tab</div>
        </div>
        <div style="background-color: white; padding: 15px; border-radius: 5px;">
            <div style="background-color: #fff3cd; color: #856404; padding: 10px; border-radius: 5px; margin-bottom: 15px;">
                ⚠️ These predictions are based on mock data and are for demonstration purposes only.
            </div>
            
            <h4>Predictions & Forecasts</h4>
            
            <div style="display: flex; justify-content: space-between;">
                <div style="flex: 1; margin-right: 10px;">
                    <h5>Predicted Future Interests</h5>
                    <div style="border: 1px solid #ddd; border-radius: 5px; padding: 10px; margin-bottom: 10px;">
                        <p><b>Artificial Intelligence</b></p>
                        <div style="display: flex; align-items: center;">
                            <span style="width: 80px;">Confidence:</span>
                            <div style="background-color: #e9ecef; height: 15px; width: 100px; border-radius: 10px; margin: 0 10px;">
                                <div style="background-color: #007bff; height: 100%; width: 85%; border-radius: 10px;"></div>
                            </div>
                            <span>85%</span>
                        </div>
                        <p><small>Reasoning: User has been increasingly engaging with AI-related content</small></p>
                    </div>
                    
                    <div style="border: 1px solid #ddd; border-radius: 5px; padding: 10px;">
                        <p><b>Sustainable Energy</b></p>
                        <div style="display: flex; align-items: center;">
                            <span style="width: 80px;">Confidence:</span>
                            <div style="background-color: #e9ecef; height: 15px; width: 100px; border-radius: 10px; margin: 0 10px;">
                                <div style="background-color: #007bff; height: 100%; width: 70%; border-radius: 10px;"></div>
                            </div>
                            <span>70%</span>
                        </div>
                        <p><small>Reasoning: Recent discussions about climate change</small></p>
                    </div>
                </div>
                
                <div style="flex: 1;">
                    <h5>Demographic Predictions</h5>
                    <div style="border: 1px solid #ddd; border-radius: 5px; padding: 10px; margin-bottom: 10px;">
                        <p>Age Range: <b>25-34</b> (Confidence: 75%)</p>
                        <p>Education Level: <b>Bachelor's Degree</b> (Confidence: 80%)</p>
                        <p>Occupation: <b>Technology Sector</b> (Confidence: 85%)</p>
                    </div>
                    
                    <h5>Brand Affinities</h5>
                    <div style="border: 1px solid #ddd; border-radius: 5px; padding: 10px;">
                        <p><b>Technology:</b> Apple, Google, Microsoft</p>
                        <p><b>Media:</b> Netflix, Spotify, YouTube</p>
                    </div>
                </div>
            </div>
        </div>
    </div>
    """
    display(HTML(html))

show_predictions_preview()

## Creating Visualizations

Let's create some simple visualizations of our results:

In [ ]:
# Create a simple bar chart of personality traits
if 'personality_traits' in results['content_analysis']:
    traits = results['content_analysis']['personality_traits']
    
    # Convert to DataFrame for easier plotting
    traits_df = pd.DataFrame(list(traits.items()), columns=['Trait', 'Score'])
    traits_df = traits_df.sort_values('Score', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Score', y='Trait', data=traits_df)
    plt.title('Personality Traits Analysis')
    plt.xlim(0, 1)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize authenticity components if available
if 'authenticity_analysis' in results and 'overall_authenticity' in results['authenticity_analysis']:
    auth = results['authenticity_analysis']['overall_authenticity']
    
    # Create a pie chart
    labels = ['Authentic', 'Risk']
    sizes = [auth['score'], 1 - auth['score']]
    colors = ['#66b3ff', '#ff9999']
    explode = (0.1, 0)  # explode the first slice for emphasis
    
    plt.figure(figsize=(8, 8))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.title('Profile Authenticity Analysis')
    plt.show()

## Fixed Desktop UI Tabs: Overview

We've recently made improvements to the ProfileScope desktop application's UI tabs to properly display analysis data. Here's what was fixed:

1. **Summary Tab**: Now properly displays profile overview, key metrics, and activity data
2. **Authenticity Tab**: Fixed to show the authenticity gauge, assessment summary, and risk factors
3. **Predictions Tab**: Now correctly displays future interests, demographic predictions, and brand affinities

The UI rendering was updated to correctly parse the mock data and display it in a user-friendly format across all tabs.

## Exporting Results

Let's export our analysis results for later use or sharing:

In [ ]:
# Create output directory
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)

# Generate output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"{output_dir}/{platform}_{profile_id}_{timestamp}.json"

# Export the results
analyzer.export_results(results, output_file)
print(f"Results exported to: {output_file}")

## Project Maintenance

ProfileScope includes utilities for project maintenance, such as the cleanup script. Here's how to use it:

In [ ]:
# Example of running the cleanup script from Python
import subprocess

def run_cleanup(verbose=True):
    """Run the ProfileScope cleanup script"""
    script_path = os.path.join(os.path.abspath('..'), 'scripts', 'cleanup.sh')
    
    if not os.path.exists(script_path):
        print(f"Cleanup script not found at: {script_path}")
        return False
    
    try:
        result = subprocess.run(['bash', script_path], 
                               capture_output=True, 
                               text=True, 
                               check=True)
        if verbose:
            print("Cleanup completed successfully")
            print("Output:")
            print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error running cleanup script: {e}")
        if verbose:
            print("Error output:")
            print(e.stderr)
        return False

# Uncomment to run cleanup (use carefully!)
# run_cleanup()

### Running Tests with Automatic Cleanup

ProfileScope now supports automatically running the cleanup script before tests. Let's see how to use it:

In [ ]:
# Run tests with automatic cleanup
def run_tests_with_cleanup():
    """Run tests with automatic cleanup first"""
    script_path = os.path.join(os.path.abspath('..'), 'run_tests.py')
    
    if not os.path.exists(script_path):
        print(f"Test runner script not found at: {script_path}")
        return False
    
    try:
        print("Running tests with automatic cleanup...")
        result = subprocess.run([sys.executable, script_path], 
                              capture_output=True, 
                              text=True)
        
        print("Test output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
            return True
        else:
            print("\n❌ Some tests failed.")
            if result.stderr:
                print("Error output:")
                print(result.stderr)
            return False
    except Exception as e:
        print(f"Error running tests: {e}")
        return False

# Uncomment to run tests with automatic cleanup
# run_tests_with_cleanup()

### Preventing Duplicate Cleanup Runs

Our testing system now prevents cleanup from running twice by using an environment variable to track whether cleanup has already run:

In [ ]:
# Example of checking and managing cleanup status
def check_cleanup_status():
    """Check and manage cleanup status"""
    if os.environ.get("PROFILESCOPE_CLEANUP_RUN") == "1":
        print("Cleanup has already been run in this session")
        return True
    else:
        print("Cleanup has not been run yet")
        return False

def reset_cleanup_status():
    """Reset cleanup status to force running it again"""
    if "PROFILESCOPE_CLEANUP_RUN" in os.environ:
        del os.environ["PROFILESCOPE_CLEANUP_RUN"]
        print("Cleanup status reset - will run again on next test")
    else:
        print("Cleanup status was not set")

# Check current cleanup status
check_cleanup_status()

### Using Alternative Test Runners

In case you encounter issues with pytest plugins like pytest-html, we provide alternative test runners:

In [ ]:
# Run tests with simple XML report (more reliable than HTML reports)
def run_simple_tests():
    """Run tests with simple XML reporting"""
    script_path = os.path.join(os.path.abspath('..'), 'run_simple_tests.py')
    
    if not os.path.exists(script_path):
        print(f"Simple test runner script not found at: {script_path}")
        return False
    
    try:
        print("Running simplified tests...")
        result = subprocess.run([sys.executable, script_path], 
                              capture_output=True, 
                              text=True)
        
        print("Test output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
            return True
        else:
            print("\n❌ Some tests failed.")
            if result.stderr:
                print("Error output:")
                print(result.stderr)
            return False
    except Exception as e:
        print(f"Error running tests: {e}")
        return False

# Run tests directly with pytest
def run_direct_pytest():
    """Run tests directly with pytest"""
    try:
        print("Running pytest directly...")
        # First run cleanup
        if not check_cleanup_status():
            run_cleanup(verbose=False)
        
        # Run pytest directly
        result = subprocess.run([sys.executable, '-m', 'pytest'], 
                              capture_output=True, 
                              text=True,
                              cwd=os.path.abspath('..'))
        
        print("Test output:")
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
            return True
        else:
            print("\n❌ Some tests failed.")
            if result.stderr:
                print("Error output:")
                print(result.stderr)
            return False
    except Exception as e:
        print(f"Error running tests: {e}")
        return False

# Use run_direct_pytest() or run_simple_tests() if you encounter issues with the standard test runner

## Conclusion

In this notebook, we've demonstrated how to:

1. Set up and initialize the ProfileScope analyzer
2. Perform a basic profile analysis
3. Explore and visualize the results
4. Examine how each UI tab in the desktop application works
5. Preview the recently fixed UI tabs with proper data display
6. Export the results for sharing or further analysis
7. Perform maintenance tasks like cleanup
8. Run tests with automatic cleanup
9. Manage cleanup run status to prevent duplicate runs
10. Use alternative test runners for better compatibility

The recent UI fixes ensure that all tabs (Summary, Authenticity, and Predictions) now properly display data and provide a comprehensive view of the analysis results. Check the desktop application to see these improvements in action.